In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorflow as tf
from sciann import Functional, Variable, SciModel, PDE
from sciann.utils import *
import scipy.io 
import time
import random

tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

---------------------- SCIANN 0.7.0.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.11.0 
Python Version: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)] 



# Model specifications

In [5]:
# Space parameters
nx, nz = 240, 240
dx, dz = 0.01, 0.01
x = np.arange(0, nx*dx, dx)
z = np.arange(0, nz*dz, dz)

pad = 10
center = [nz / 2, nx / 2] # center of velmodel
x0 = nx / 2 * dx 
z0 = nz / 2 * dz # x and z coordinates of the center of the velmodel
radius = 1

r = 1.83/2 # radius of a smaller circular region
n = 8 # number of receivers and sources
tet = np.linspace(-np.pi, np.pi, n + 1) # array of angles for placing the points around the circle
xi = r * np.cos(tet) + x0 # x-coordinates of sources and receivers
zi = r * np.sin(tet) + z0 # z-coordinates of sources and receivers

xi = xi[:-1]  # Slices the array to remove the last element
zi = zi[:-1]  # Slices the array to remove the last element

## Source location

In [7]:
sx = np.round(xi / dx).astype(int)  # x-coordinates of sources and receivers in grid points
sz = np.round(zi / dz).astype(int)  # z-coordinates of sources and receivers in grid points
Ns = len(sx) # number of sources
s = [sz.T, sx.T] # source and receiver coordinates

## Receiver positions

In [10]:
rx = sx
rz = sz
r = s
Nr = len(rx) # number of receivers

## Source-Receiver distances

In [12]:
distances = np.zeros((Nr, Ns))
for i in range(1, Ns):
    distances[i, :] = np.sqrt((rx - sx[i])**2 * dx**2 + (rz - sz[i])**2 * dz**2)

## Time parameters of observed data

In [15]:
tmax = 1/1000
dts = 1/500000
ts = np.arange(0, tmax + dts, dts) # time axis
nts = len(ts)

## Filtering and normalizing the data

In [16]:
lapalacian = 2

filt = [20, 30, 60, 70] * 1000

fmin = [filt[1], filt[1] - filt[0]]
fmax = [filt[2], filt[3] - filt[2]]
phase = 0
max_atten = 80 # maximum attenuation in dB

## Windowing

# Source

# Reference Receiver for shots

# Filtering and Normalizing the data